# Optimizing model parameters

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="/home/hslee/Desktop/Datasets/",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="/home/hslee/Desktop/Datasets/",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [3]:
class NeuralNetwork(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
    
    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()

# Hyperparameters

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Optimization Loop

## Loss Function

In [7]:
loss_fn = nn.CrossEntropyLoss()

## Optimizer

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Train loop

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer) :
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader) :
        # Compute prediction and loss 
        pred = model(X) # model.forward(X)을 직접 호출해줌
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0 :
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn) :
    size = len(dataloader.dataset)
    model.eval()
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad() :
        for X, y in dataloader :
            pred = model.forward(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error : \n Accuracy : {(100*correct):>0.1f}%, Avg loss : {test_loss:>8f}\n")
                
        

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302387 [    0/60000]
loss: 2.294538 [ 6400/60000]
loss: 2.276784 [12800/60000]
loss: 2.273392 [19200/60000]
loss: 2.266725 [25600/60000]
loss: 2.248604 [32000/60000]
loss: 2.258590 [38400/60000]
loss: 2.232116 [44800/60000]
loss: 2.249221 [51200/60000]
loss: 2.190337 [57600/60000]
Test Error : 
 Accuracy : 39.5%, Avg loss : 2.201375

Epoch 2
-------------------------------
loss: 2.227686 [    0/60000]
loss: 2.215777 [ 6400/60000]
loss: 2.170330 [12800/60000]
loss: 2.173150 [19200/60000]
loss: 2.124080 [25600/60000]
loss: 2.104861 [32000/60000]
loss: 2.116305 [38400/60000]
loss: 2.066200 [44800/60000]
loss: 2.129580 [51200/60000]
loss: 1.990629 [57600/60000]
Test Error : 
 Accuracy : 50.0%, Avg loss : 2.025319

Epoch 3
-------------------------------
loss: 2.095104 [    0/60000]
loss: 2.058726 [ 6400/60000]
loss: 1.989639 [12800/60000]
loss: 1.994377 [19200/60000]
loss: 1.872549 [25600/60000]
loss: 1.878177 [32000/60000]
loss: 1.886233 [38